In [1]:
from dotenv import load_dotenv
import json
import html
import re
import time
import httpx

load_dotenv()

True

In [ ]:
from enum import Enum

BASE_URL = "http://localhost:8000"

class agent_type_enum(Enum):
    agent_no_tools = "agent_no_tools"
    agent_client_tools = "agent_client_tools"
    agent_all_raw = "agent_all_raw"
    agent_all_xml = "agent_all_xml"
    
def call_agent_api(
    prompt: str | list[dict] | dict, 
    agent_uuid: str = None, 
    agent_type: str = "agent_all_raw", 
    log_file: str = None
) -> str:
    """Call agent API and log streamed response to file.
    
    Returns:
        Path to the log file
    """
    # Build filename
    timestamp = int(time.time() * 1000)
    if log_file is None:
        log_file = f"temp_logs/{agent_type}_{timestamp}.log"
    else:
        log_file = f"temp_logs/{log_file}_{agent_type}_{timestamp}.log"
    
    # Build request payload
    payload = {"user_prompt": prompt}
    if agent_uuid:
        payload["agent_uuid"] = agent_uuid
    else:
        payload["agent_type"] = agent_type
    
    # Stream response and write to file
    with open(log_file, "w") as f:
        with httpx.Client(timeout=None) as client:
            with client.stream("POST", f"{BASE_URL}/agent/run", json=payload) as response:
                for line in response.iter_lines():
                    if line.startswith("data: "):
                        data = line[6:]  # Strip "data: " prefix
                        if data != "[DONE]":
                            # Unescape newlines and write
                            f.write(data.replace("\\n", "\n"))
                            f.write("\n")
    
    return log_file

In [8]:
path = call_agent_api("Hello", log_file="test")
path

'temp_logs/test_agent_all_raw_1766658756005.log'

# Raw formatted Output

## No tools

In [ ]:
call_agent_api("Hi, what is AI?", log_file="test", agent_type="agent_no_tools")

## Client Tools

In [ ]:
call_agent_api("Hi, what is 5*(3+2/4)?", log_file="test", agent_type="agent_client_tools")

## Server Tools

In [ ]:
call_agent_api("Hi, what is 5*(3+2/4)?", log_file="test", agent_type="agent_all_raw")

In [ ]:
prompt = {
    
}
call_agent_api("Hi, what is ?", log_file="test", agent_type="agent_all_raw")

## PDF citations

## Content citations

## Complex markdown Rendering Cases

## Image/File Generation

# XML Formatted Output

## No tools

## Client Tools

## Server Tools

## PDF citations

## Content citations

## Complex markdown Rendering Cases

## Image/File Generation